In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)

In [ ]:
openai = OpenAI(
    base_url=os.getenv("OPENROUTER_API_URL"), api_key=os.getenv("OPENROUTER_API_KEY")
)

In [ ]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

resume

In [ ]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

summary

In [ ]:
name = "Rahul Dey"

In [ ]:
system_prompt = (
    f"You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s career, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and resume which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer, say so. "
)

system_prompt += f"\n\n## Summary: \n {summary}\n\n## Resume: \n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

print(system_prompt)

In [ ]:
def chat(message, history):
    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )
    response = openai.chat.completions.create(
        model="openai/gpt-oss-20b:free", messages=messages
    )
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat).launch()

In [ ]:
from pydantic import BaseModel


class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
    You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
    The Agent is playing the role of {name} and is representing {name} on their website. \
    The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    The Agent has been provided the context on {name} in the form of their summary and resume details. Here is the information:"

evaluator_system_prompt += f"\n\n## Summary: \n{summary}\n\n## Resume: \n{resume}\n\n"
evaluator_system_prompt += "With this context, please evaluate the latest response, replying with weather the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history) -> str:
    user_prompt = (
        f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    )
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with weather it is acceptable and your feedback."
    return user_prompt

In [ ]:
def evaluate(reply, message, history) -> Evaluation | None:
    user_prompt = evaluator_user_prompt(reply, message, history)
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = openai.beta.chat.completions.parse(
        model="google/gemma-3-27b-it:free",
        messages=messages,  # type: ignore
        response_format=Evaluation,
    )
    return response.choices[0].message.parsed

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Do you hold a patent?"},
]
response = openai.chat.completions.create(
    model="openai/gpt-oss-120b:free",
    messages=messages,  # type: ignore
)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply=reply, message="Do you hold a patent?", history=messages[:1])